#### Visualization of Xenium add-on panel healthy gut data after the labels transer 
- **Developed by:** Anna Maguza
- **Affilation:** Faculty of Medicine, Würzburg University
- **Created date:** 20th March 2024
- **Last modified date:** 13th October 2024

##### Import packages

In [2]:
import anndata as ad
import scanpy as sc
import squidpy as sq
import pandas as pd
from scipy.io import mmread
import matplotlib.pyplot as plt
import numpy as np
import scipy as sci
from scipy.sparse import coo_matrix
import seaborn as sns

#### Set up working environment

In [3]:
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 180, color_map = 'magma_r', dpi_save = 300, vector_friendly = True, format = 'svg')

-----
anndata     0.10.5.post1
scanpy      1.9.8
-----
PIL                         10.2.0
anyio                       NA
arrow                       1.3.0
asciitree                   NA
asttokens                   NA
attr                        23.2.0
attrs                       23.2.0
babel                       2.14.0
backcall                    0.2.0
beta_ufunc                  NA
binom_ufunc                 NA
brotli                      1.1.0
certifi                     2023.11.17
cffi                        1.16.0
charset_normalizer          3.3.2
cloudpickle                 3.0.0
comm                        0.2.1
cycler                      0.12.1
cython_runtime              NA
dask                        2024.1.1
dask_image                  2023.08.1
datashader                  0.16.0
datatree                    0.0.13
dateutil                    2.8.2
debugpy                     1.8.0
decorator                   5.1.1
defusedxml                  0.7.1
docrep                   

#### Data upload

+ Load h5ad object

In [4]:
adata_xenium = sc.read_h5ad('data/10X_Xenium_adult_colon_data/10X_Xenium_add-on_without_image.h5ad')

#### Visualise predicted cell types - after cell types + Stem cells labels transfer

+ Upload the object with cell types

In [5]:
adata_full = sc.read_h5ad('data/10X_Xenium_adult_colon_data/Xenium_add_on_data_scANVI_cell_type_and_stem_cells.h5ad')

In [6]:
adata_full.obs.index.name = 'cell_id'
adata_xenium.obs.index.name = 'cell_id'
del adata_xenium.obs['cell_id']

In [7]:
adata_xenium.obs = adata_xenium.obs.merge(adata_full.obs[['C_scANVI']], on='cell_id', how='left')
adata_xenium.obs.rename(columns={'C_scANVI':'Cell_Type'}, inplace=True)

+ Visualize the cell types with standard color pallet

In [7]:
sc.set_figure_params(scanpy=True, dpi=300, figsize=(10, 10))
sq.pl.spatial_scatter(
    adata_xenium,
    library_id="spatial",
    img=True, img_res_key='lowres',
    shape=None,
    color=[
        "Cell_Type",
    ],
    size=0.1,
    alpha=1
)

In [9]:
adata_xenium.uns['Cell_Type_colors']

['#1f77b4',
 '#ff7f0e',
 '#279e68',
 '#d62728',
 '#aa40fc',
 '#8c564b',
 '#e377c2',
 '#b5bd61',
 '#17becf']

+ Change the color pallet to highlight stem cells

In [10]:
# Define your custom color palette
custom_colors = [
    "#41729F",  # B cells
    "#A16AE8",  # Endothelial
    "#B6E5D8",  # Epithelial
    "#EC9EC0",  # Mesenchymal + 
    "#FBE5C8",  # Myeloid + 
    "#4D5D53",  # Neuronal + 
    "#870052",  # Plasma cells + 
    "#FF0000FF",  # Stem cells + 
    "#ffde17"   # T cells +
]

# Update the color codes in the AnnData object
adata_xenium.uns['Cell_Type_colors'] = custom_colors

# Now, when you plot using sq.pl.spatial_scatter, it should use your updated custom colors
sq.pl.spatial_scatter(
    adata_xenium,
    library_id="spatial",
    img=True,
    img_res_key='lowres',
    shape=None,
    color="Cell_Type",
    size=0.1,
    alpha=1
)

+ Add stem cells types

In [8]:
adata_stem_cells = sc.read_h5ad('data/10X_Xenium_adult_colon_data/Stem_cells_scRNA_and_Xenium_add_on_scANVI_7K_cells.h5ad')

In [9]:
adata_stem_cells = adata_stem_cells[adata_stem_cells.obs['Study_name'] == '10X_Xenium_gut_data']
adata_stem_cells = adata_stem_cells[adata_stem_cells.obs.index.isin(adata_xenium.obs.index)]

In [10]:
adata_xenium.obs['Stem_cells'] = 'Unknown'
adata_xenium.obs.loc[adata_stem_cells.obs.index, 'Stem_cells'] = adata_stem_cells.obs['C_scANVI']

In [11]:
adata_xenium.obs['CellType2'] = adata_xenium.obs['Cell_Type'].copy()
adata_xenium.obs['CellType2'] = adata_xenium.obs['CellType2'].cat.add_categories(['MTRNR2L12+ASS1+_SC',
                                                                                  'RPS10+_RPS17+_SC',
                                                                                  'FXYD3+_CKB+_SC'])

adata_xenium.obs.loc[(adata_xenium.obs['Stem_cells'] == 'MTRNR2L12+ASS1+_SC'), 'CellType2'] = 'MTRNR2L12+ASS1+_SC'
adata_xenium.obs.loc[(adata_xenium.obs['Stem_cells'] == 'RPS10+_RPS17+_SC'), 'CellType2'] = 'RPS10+_RPS17+_SC'
adata_xenium.obs.loc[(adata_xenium.obs['Stem_cells'] == 'FXYD3+_CKB+_SC'), 'CellType2'] = 'FXYD3+_CKB+_SC'

In [12]:
adata_xenium.obs['CellType2'] = adata_xenium.obs['CellType2'].cat.remove_unused_categories()

+ Visualize cell types + stem cells populations with the standard color pallet

In [16]:
sc.set_figure_params(scanpy=True, dpi=300, figsize=(10, 10))
sq.pl.spatial_scatter(
    adata_xenium,
    library_id="spatial",
    img=True, img_res_key='lowres',
    shape=None,
    color=[
        "CellType2",
    ],
    size=0.1,
    alpha=1
)

+ Change color pallete

In [17]:
# Define your custom color palette
custom_colors = [
    "#41729F",  # B cells
    "#A16AE8",  # Endothelial
    "#B6E5D8",  # Epithelial
    "#EC9EC0",  # Mesenchymal + 
    "#FBE5C8",  # Myeloid + 
    "#4D5D53",  # Neuronal + 
    "#870052",  # Plasma cells +  
    "#ffde17",   # T cells +
    "#00B1B0",  # Stem cells MTRN
    "#508FF7",  # Stem cells RPS
    "#FF0000FF"  # Stem cells FXYD
]

# Update the color codes in the AnnData object
adata_xenium.uns['CellType2_colors'] = custom_colors

# Now, when you plot using sq.pl.spatial_scatter, it should use your updated custom colors
sq.pl.spatial_scatter(
    adata_xenium,
    library_id="spatial",
    img=True,
    img_res_key='lowres',
    shape=None,
    color="CellType2",
    size=0.1,
    alpha=1
)

In [ ]:
adata_xenium.write_h5ad('data/10X_Xenium_adult_colon_data/Xenium_add_on_celltypes_and_stem_cells_types_without_image.h5ad')

+ Visualise only Stem cells

In [ ]:
custom_colors = [
    "#41729F",  # B cells
    "#A16AE8",  # Endothelial
    "#B6E5D8",  # Epithelial
    "#EC9EC0",  # Mesenchymal + 
    "#FBE5C8",  # Myeloid + 
    "#4D5D53",  # Neuronal + 
    "#870052",  # Plasma cells +  
    "#ffde17",   # T cells +
    "#00B1B0",  # Stem cells MTRN
    '#FFDB15',  # Stem cells RPS
    "#FF0000FF"  # Stem cells FXYD
]

# Update the color codes in the AnnData object
adata_xenium.uns['CellType2_colors'] = custom_colors

In [ ]:
sc.set_figure_params(scanpy=True, dpi=300, figsize=(20, 10))

cell_types_of_interest = ['MTRNR2L12+ASS1+_SC', 'RPS10+_RPS17+_SC', 'FXYD3+_CKB+_SC']

stem_cells_mask = adata_xenium.obs['CellType2'].isin(cell_types_of_interest)

In [ ]:
adata_stem_cells = adata_xenium[stem_cells_mask, :]

In [ ]:
sq.pl.spatial_scatter(
    adata_stem_cells,  # Use the filtered AnnData object.
    library_id="spatial",
    img=True,
    shape=None,  # Assuming you want the default shape.
    color="CellType2",  # This will now only show 'Stem cells'.
    size=0.1,
    alpha=1
)

+ Add other cell states

In [13]:
adata_scvi = sc.read_h5ad('data/10X_Xenium_adult_colon_data/Xenium_add_on_data_scANVI_cell_states.h5ad')

In [14]:
adata_scvi.obs.index.name = 'cell_id'

In [15]:
adata_xenium.obs = adata_xenium.obs.merge(adata_scvi.obs[['C_scANVI']], on='cell_id', how='left')

In [16]:
adata_xenium.obs['C_scANVI'] = adata_xenium.obs['C_scANVI'].cat.add_categories(['MTRNR2L12+ASS1+_SC',
                                                                                  'RPS10+_RPS17+_SC',
                                                                                  'FXYD3+_CKB+_SC'])

adata_xenium.obs.loc[(adata_xenium.obs['Stem_cells'] == 'MTRNR2L12+ASS1+_SC'), 'C_scANVI'] = 'MTRNR2L12+ASS1+_SC'
adata_xenium.obs.loc[(adata_xenium.obs['Stem_cells'] == 'RPS10+_RPS17+_SC'), 'C_scANVI'] = 'RPS10+_RPS17+_SC'
adata_xenium.obs.loc[(adata_xenium.obs['Stem_cells'] == 'FXYD3+_CKB+_SC'), 'C_scANVI'] = 'FXYD3+_CKB+_SC'

In [17]:
df = adata_xenium.obs['C_scANVI'].value_counts()

cell_types_to_keep = df[df >= 10].index

adata_xenium = adata_xenium[adata_xenium.obs['C_scANVI'].isin(cell_types_to_keep), :]

In [18]:
adata_xenium.obs.loc[(adata_xenium.obs['C_scANVI'] == 'B cells AICDA LRMP'), 'C_scANVI'] = 'B cells'
adata_xenium.obs.loc[(adata_xenium.obs['C_scANVI'] == 'Naive B'), 'C_scANVI'] = 'B cells'
adata_xenium.obs.loc[(adata_xenium.obs['C_scANVI'] == 'Memory B'), 'C_scANVI'] = 'B cells'
adata_xenium.obs.loc[(adata_xenium.obs['C_scANVI'] == 'Enterocyte'), 'C_scANVI'] = 'Colonocyte'

adata_xenium.obs['C_scANVI'] = adata_xenium.obs['C_scANVI'].cat.add_categories(['Arterial capillary',
                                                                                  'Gamma delta T cells'])
adata_xenium.obs.loc[(adata_xenium.obs['C_scANVI'] == 'arterial capillary'), 'C_scANVI'] = 'Arterial capillary'
adata_xenium.obs.loc[(adata_xenium.obs['C_scANVI'] == 'gdT'), 'C_scANVI'] = 'Gamma delta T cells'

/tmp/ipykernel_1636067/3728328697.py:6: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_xenium.obs['C_scANVI'] = adata_xenium.obs['C_scANVI'].cat.add_categories(['Arterial capillary',


In [19]:
adata_xenium.obs['C_scANVI'] = adata_xenium.obs['C_scANVI'].cat.remove_unused_categories()

+ Add deep crypt secretory cells

In [20]:
adata_xenium_log = adata_xenium.copy()
sc.pp.normalize_total(adata_xenium_log, target_sum = 1e6, exclude_highly_expressed = True)
sc.pp.log1p(adata_xenium_log)

normalizing counts per cell The following highly-expressed genes are not considered during normalization factor computation:
['ABCA8', 'ABCC8', 'ACACB', 'ACKR1', 'ACTA2', 'ADAM28', 'ADH1C', 'ADRA2A', 'AFAP1L2', 'AGTR1', 'AKR1C3', 'AKR7A3', 'ALDH1B1', 'ANK2', 'ANO7', 'ANPEP', 'ANXA1', 'ANXA13', 'APOB', 'APOE', 'AQP1', 'AQP8', 'AREG', 'ARHGAP24', 'ARX', 'ASCL2', 'ATOH1', 'AVIL', 'AZGP1', 'B3GNT6', 'BANK1', 'BATF', 'BCAS1', 'BEST2', 'BEST4', 'BMP4', 'BMP5', 'BMX', 'BRCA2', 'C1QA', 'C1QB', 'C1QBP', 'C1QC', 'C2orf88', 'C7', 'CA1', 'CA2', 'CA4', 'CA7', 'CADM2', 'CADPS', 'CALB2', 'CCDC80', 'CCL11', 'CCL13', 'CCL20', 'CCL4', 'CCL5', 'CCR7', 'CD14', 'CD163', 'CD177', 'CD2', 'CD24', 'CD36', 'CD3D', 'CD3E', 'CD3G', 'CD40LG', 'CD5', 'CD6', 'CD7', 'CD79A', 'CD79B', 'CD83', 'CD8A', 'CD8B', 'CDCA7', 'CDH1', 'CDH19', 'CDHR5', 'CDK15', 'CDK6', 'CDKN2B', 'CEACAM1', 'CEACAM5', 'CEACAM6', 'CEACAM7', 'CEP126', 'CES1', 'CES2', 'CFTR', 'CHGA', 'CHGB', 'CHI3L2', 'CHP2', 'CHRM3', 'CKAP4', 'CLCA1', 'CLCA4', 'CL

/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:196: UserWarning: Some cells have zero counts
  warn(UserWarning('Some cells have zero counts'))


In [21]:
genes = ['REG4']
sc.tl.score_genes(adata_xenium_log, genes, score_name = 'REG4_score')

adata_xenium_log.obs['Cell_States'] = adata_xenium_log.obs['C_scANVI'].copy()
adata_xenium_log.obs['Cell_States'] = adata_xenium_log.obs['Cell_States'].cat.add_categories('Deep crypt secretory cells')
adata_xenium_log.obs.loc[adata_xenium_log.obs['REG4_score'] > 14, 'Cell_States'] = 'Deep crypt secretory cells' 

computing score 'REG4_score'
    finished: added
    'REG4_score', score of gene set (adata.obs).
    17 total control genes are used. (0:00:00)


In [22]:
categories = sorted(adata_xenium_log.obs['Cell_States'].unique())

In [23]:
adata_xenium_log.obs['Cell_States'] = pd.Categorical(
    adata_xenium_log.obs['Cell_States'],
    categories=categories,
    ordered=True
)

+ Visualize with standard color map

In [24]:
with plt.rc_context():
    sc.set_figure_params(dpi = 300, figsize=(20,10))
    sq.pl.spatial_scatter(
    adata_xenium_log,
    library_id="spatial",
    img=True,
    img_res_key='lowres',
    shape=None,
    color="Cell_States",
    size=0.1,
    alpha=1
)
    plt.savefig("data/10X_Xenium_adult_colon_data/Xenium_cell_annotation.png", bbox_inches="tight")

In [62]:
with plt.rc_context():
    sc.set_figure_params(dpi = 300, figsize=(20,10))
    sq.pl.spatial_scatter(
    adata_xenium_log,
    library_id="spatial",
    img=True,
    img_res_key='lowres',
    shape=None,
    color="LGR5",
    size=0.1,
    alpha=1
)
    plt.savefig("data/10X_Xenium_adult_colon_data/LGR5_expression.png", bbox_inches="tight")

In [ ]:
with plt.rc_context():
    sc.set_figure_params(dpi = 300, figsize=(20,10))
    sq.pl.spatial_scatter(
    adata_xenium_log,
    library_id="spatial",
    img=True,
    img_res_key='lowres',
    shape=None,
    color="SMOC2",
    size=0.1,
    alpha=1
)
    plt.savefig("data/10X_Xenium_adult_colon_data/SMOC2_expression.png", bbox_inches="tight")

+ Visualize only fetal SC subsets

In [66]:
# Define the custom color mapping
custom_colors = {
    'MTRNR2L12+ASS1+_SC': '#ba0900',
    'RPS10+_RPS17+_SC': '#0000a6',
    'FXYD3+_CKB+_SC': '#ff34ff'
}

# Create a color list where all categories are light grey except the specified ones
all_categories = adata_xenium_log.obs['Cell_States'].cat.categories
color_list = ['#D3D3D3' if cat not in custom_colors else custom_colors[cat] for cat in all_categories]

# Update the color mapping in the AnnData object
adata_xenium_log.uns['Cell_States_colors'] = color_list

with plt.rc_context():
    sc.set_figure_params(scanpy=True, dpi=300, figsize=(20, 10))
    sq.pl.spatial_scatter(
    adata_xenium_log,
    library_id="spatial",
    img=True,
    img_res_key='lowres',
    shape=None,
    color="Cell_States",
    size=0.1,
    alpha=1
)
    plt.savefig("data/10X_Xenium_adult_colon_data/only_fetalSC_subsets.png", bbox_inches="tight")


+ Calculate neighborhood enrichment

In [25]:
sq.gr.spatial_neighbors(adata_xenium_log, coord_type="generic", delaunay=True)
sq.gr.nhood_enrichment(adata_xenium_log, cluster_key="Cell_States") 

Creating graph using `generic` coordinates and `None` transform and `1` libraries.
Adding `adata.obsp['spatial_connectivities']`
       `adata.obsp['spatial_distances']`
       `adata.uns['spatial_neighbors']`
Finish (0:00:31)
Calculating neighborhood enrichment using `1` core(s)


100%|███████████████████████████████████████████████████| 1000/1000 [00:24<00:00, 40.18/s]

Adding `adata.uns['Cell_States_nhood_enrichment']`
Finish (0:00:24)


In [26]:
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(15, 15))
    sq.pl.nhood_enrichment(adata_xenium_log, cluster_key="Cell_States", figsize=(15, 15), cmap="RdYlBu_r")
    plt.savefig("data/10X_Xenium_adult_colon_data/nhood_enrichment.png", bbox_inches="tight")
    plt.close()

/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/pl/_utils.py:556: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row_labels = adata.obs[key][row_order]


In [28]:
sq.gr.co_occurrence(adata_xenium_log, cluster_key="Cell_States")
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(4.5, 5))
    sq.pl.co_occurrence(adata_xenium_log, cluster_key="Cell_States", clusters="FXYD3+_CKB+_SC", figsize=(10, 10))
    plt.savefig("data/10X_Xenium_adult_colon_data/co_occurrence_FXYD3+_CKB+_SC.png", bbox_inches="tight")
    plt.close() 

Calculating co-occurrence probabilities for `50` intervals `9045` split combinations using `1` core(s)


100%|███████████████████████████████████████████████████| 9045/9045 [31:14<00:00,  4.83/s]


Adding `adata.uns['Cell_States_co_occurrence']`
Finish (0:31:14)


In [33]:
sq.gr.co_occurrence(adata_xenium_log, cluster_key="Cell_States")
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(4.5, 5))
    sq.pl.co_occurrence(adata_xenium_log, cluster_key="Cell_States", clusters="MTRNR2L12+ASS1+_SC", figsize=(10, 10))
    plt.savefig("data/10X_Xenium_adult_colon_data/co_occurrence_MTRNR2L12+ASS1+_SC.png", bbox_inches="tight")
    plt.close() 

Calculating co-occurrence probabilities for `50` intervals `9045` split combinations using `1` core(s)


100%|███████████████████████████████████████████████████| 9045/9045 [31:16<00:00,  4.82/s]


Adding `adata.uns['Cell_States_co_occurrence']`
Finish (0:31:17)


In [34]:
sq.gr.co_occurrence(adata_xenium_log, cluster_key="Cell_States")
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(4.5, 5))
    sq.pl.co_occurrence(adata_xenium_log, cluster_key="Cell_States", clusters="RPS10+_RPS17+_SC", figsize=(10, 10))
    plt.savefig("data/10X_Xenium_adult_colon_data/co_occurrence_RPS10+_RPS17+_SC.png", bbox_inches="tight")
    plt.close() 

Calculating co-occurrence probabilities for `50` intervals `9045` split combinations using `1` core(s)


: 